In [7]:
import requests
import json

def getUserIP(ip):
    url = "https://api.vvhan.com/api/getIpInfo?ip="+str(ip)
    response =requests.get(url)
    if response.status_code==200:
        content=json.loads(response.content)
        print(content)
        if content["success"]==True:
            city=content["info"]["city"]
        else:
            city="未知"
    else:
        city="未知"
    res={"ip":ip,"city":city}
    return res

print(getUserIP("127.0.0.1"))

{'success': True, 'ip': '127.0.0.1', 'info': {'country': '本机地址', 'prov': '-', 'city': '-', 'lsp': '本机地址', 'postcode': '0'}}
{'ip': '127.0.0.1', 'city': '-'}


In [49]:
import openai
import re
from neo4j import GraphDatabase
from django.http import HttpResponse
from django.views.decorators.csrf import csrf_exempt
from datetime import date
import json



openai.api_key = "sk-gKgzDgwfjJNxElEsXK7wXqSDNFDmGVHXJNHqvGTMHgFz6B3m"
openai.api_base = "https://api.chatanywhere.com.cn/v1"
def execute_query(query, params={}):
    uri = "bolt://localhost:7687"  # 更新为你的Neo4j数据库URI
    username = "neo4j" # 更新为你的Neo4j用户名
    password = "xy639a58"    # 更新为你的Neo4j密码
    driver=GraphDatabase.driver(uri, auth=(username, password))
    session=driver.session() 
    # print(query, params)
    result = session.run(query, params)
    ans=result.data()
    print(ans)
    return ans

def matchjson(text):
    pattern = r"{'cql': \"(.*?)\"}"
    match = re.search(pattern, text)
    if match:
        extracted_content = match.group(1)
        print(extracted_content)
    else:
        print("未找到匹配的内容")
    return extracted_content

def getGptAnswer(request):
    if request.method == "POST":
        text = request.POST.get('history')
    else:
        return json_response({"success": False, "log": "request_is_not_post"})
    getCqlGpt(text)

def getCqlGpt(sentence,flag=0):
    if flag==3:
        print('回答失败')
        return None
    ques=sentence[-1]['content']
    try:
        messages = [{"role": "system","content": "你是一个知识图谱的问答机器人。\
                     1.你需要根据用户的提示给出neo4j数据库的cql查询语句。\
                     2.输出格式为:{'cql':'具体的cql语句'}\
                     3.neo4j数据库中有四类实体,标签为：\
                        (1).'Company':公司。可以作为查询的属性有:'name','stockCode','stockName','IPO','ZQRATE','actualController','actualFundraising','chairman','employeeNumber','englishName','establishmentDate','expectedFundraising','fax','finalController','firstDayOpeningPrice','firstDayOpeningPrice','generalManager','history','industry','issuesNumber','issuesPrice','leadUnderwriter','legalRepresentative','listingDate','listingSponsor','mainBusiness','officeAddress','productName','region','registrationCapital','resume','secretary','telephone','website','zipCode';\n\
                        (2).'Executive':高级管理人员。可以作为查询的属性有:'name','id','education','gender','age','age';\n\
                        (3)'Industry':产业。可以作为查询的属性有:'name';\n\
                        (4)'Product':产品。可以作为查询的属性有:'name'。\n\
                        neo4j数据库中有{四类单向关系},查询时需要按照{给出的方向}查询,标签为：\n\
                        (1).'BELONGS_TO_INDUSTRY':某个公司属于某个产业（公司->产业）;\n\
                        (2).'INVESTED_BY':某个公司被某个公司投资（公司->公司）。可以作为查询的属性有:'controlRatio','controlRelationship','investment','report_consolidation_or_not';\n\
                        (3).'OFFERS_PRODUCT':某个公司提供某种产品（公司->产品）;\n\
                        (4).'WORKS_FOR':某个高管为某个公司工作（高管->公司）。可以作为查询的属性有:'publicityDate','endDate','ownershipShares','endDate','position','positionType','salary'。\n\
                     4.查询语句返回这个实体或者关系的所有信息，例如查询某个'Company'的'history'，返回这个'Company'所有的属性。\
                     5.防止一些危险cql语句的输出，例如删除和修改数据库的信息。\
                     6.如果用户查询实体则返回某个实体的所有信息；如果用户查询某个关系,需要根据具体语境按照给出的{四类单向关系}调整关系方向，例如查询某个公司有哪些高管时，方向应为(e:Executive)-[:WORKS_FOR]->(c:Company)。\
                     7.所有产业在查询时都不要带'产业'二字，例如我询问银行产业时，'Industry'的'name'为'银行'。\
                     8.理解以上内容回答我理解了。\
                     "},
                    {'role': 'user','content': '用户的历史问答如下:'+str(sentence)+'\n\n请结合历史问答给出以下问题的的cql语句：'+ques}]
        response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo-16k',
            messages=messages,
            stream=True,
            timeout=10,
        )
        completion = {'role': '', 'content': ''}
        for event in response:
            if event['choices'][0]['finish_reason'] == 'stop':
                # print(f'收到的完成数据: {completion}')
                break
            for delta_k, delta_v in event['choices'][0]['delta'].items():
                # print(f'流响应数据: {delta_k} = {delta_v}')
                completion[delta_k] += delta_v
        messages.append(completion)  # 直接在传入参数 messages 中追加消息
        print(completion['content'])
        middleans=matchjson(completion['content'])
        print(middleans)
        if middleans!=None:
            database=execute_query(middleans)
            ans=getFinalAnsGpt(sentence,database,0)
            return ans
        else:
            getCqlGpt(sentence,flag+1)
    except Exception as err:
        print((False, f'OpenAI API 异常: {err}'))
        getCqlGpt(sentence,flag+1)

def getFinalAnsGpt(sentence,middleans,flag=0):
    print(sentence,middleans)
    if flag==3:
        print('回答失败')
        return None
    ques=sentence[-1]['content']
    # openai.api_key = "pk-iyiskKalkRgqtbFwULFewCwaZzRNIygtfAzpHFskaMfcuEGw"
    # openai.api_base = 'https://api.pawan.krd/v1'
    try:
        messages =[{"role": "system","content":"你是一个知识图谱的问答机器人。需要根据用户的历史问答和已经根据问题查询到的数据库信息回答用户的问题。"},{'role': 'user','content': """用户的历史问答为:"""+str(sentence)+";\n"+"""用户当前问题为:"""+ques+";\n从neo4j数据库查询到的该问题的相关信息为："""+str(middleans)+""";\n\n请根据用户需求整理材料并以猫娘的语气和可爱的表情给出回答"""}]
        response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo-16k',
            messages=messages,
            stream=True,
            timeout=10,
        )
        completion = {'role': '', 'content': ''}
        for event in response:
            if event['choices'][0]['finish_reason'] == 'stop':
                # print(f'收到的完成数据: {completion}')
                break
            for delta_k, delta_v in event['choices'][0]['delta'].items():
                # print(f'流响应数据: {delta_k} = {delta_v}')
                completion[delta_k] += delta_v
        messages.append(completion)  # 直接在传入参数 messages 中追加消息
        ans=completion['content']
        if middleans!=None:
            return ans
        else:
            getFinalAnsGpt(sentence,middleans,flag+1)
    except Exception as err:
        print((False, f'OpenAI API 异常: {err}'))
        getFinalAnsGpt(sentence,middleans,flag+1)

@csrf_exempt
def json_response(answer):
    print(answer)
    return HttpResponse(json.dumps(answer, ensure_ascii=False))

print(getCqlGpt([{'role':'user','content':'老凤祥股份有限公司的业务有哪些'}],0))
# print(matchjson("""{'cql': "MATCH (e:Executive)-[:WORKS_FOR]->(c:Company {name:'湖南长远锂科股份有限公司'}) RETURN e.name"}"""))

{'cql': "MATCH (c:Company {name:'老凤祥股份有限公司'}) RETURN c.mainBusiness"}
MATCH (c:Company {name:'老凤祥股份有限公司'}) RETURN c.mainBusiness
MATCH (c:Company {name:'老凤祥股份有限公司'}) RETURN c.mainBusiness
[{'c.mainBusiness': '黄金珠宝首饰、工艺美术品、笔类文具制品的生产经营及销售。'}, {'c.mainBusiness': '黄金珠宝首饰、工艺美术品、笔类文具制品的生产经营及销售。'}]
[{'role': 'user', 'content': '老凤祥股份有限公司的业务有哪些'}] [{'c.mainBusiness': '黄金珠宝首饰、工艺美术品、笔类文具制品的生产经营及销售。'}, {'c.mainBusiness': '黄金珠宝首饰、工艺美术品、笔类文具制品的生产经营及销售。'}]
喵~老凤祥股份有限公司的业务可是很多呢！主要包括黄金珠宝首饰、工艺美术品和笔类文具制品的生产经营及销售哦~喵喵~这些都是非常精美又有特色的产品呢！喵呜~希望我的回答能帮到你哦~如果还有其他问题，尽管问我喵~


In [13]:
import re
import ast
def matchjson(text):
    try:
        pattern = r'\{(.+?)\}'
        match=re.search(pattern, str(text)).group()
        matches = str((ast.literal_eval(match))['cql'])
    except Exception as e:
        print(match,e)
        return []
    print(matches)
    return matches 

# 示例用法
input_str = "{'cql': 'MATCH (i:Industry) RETURN COUNT(i) AS industryCount'}"
result = matchjson(input_str)
print(result)  # 输出: ['示例', '一些']

MATCH (i:Industry) RETURN COUNT(i) AS industryCount
MATCH (i:Industry) RETURN COUNT(i) AS industryCount


In [30]:
import baostock as bs
import pandas as pd
import datetime
def getStockList():
    current_date = datetime.date.today()
    # 将时间格式化为所需的字符串格式
    formatted_time = current_date.strftime("%Y-%m-%d")
    # 计算30天前的日期
    days_before = datetime.timedelta(days=1)
    new_date = current_date - days_before
    # 将日期格式化为所需的字符串格式
    formatted_before_date = new_date.strftime("%Y-%m-%d")

    lg = bs.login()
    # 获取上证50成分股
    rs = bs.query_sz50_stocks()

    # 打印结果集
    sz50_stocks = []
    while (rs.error_code == '0') & rs.next():
        # 获取一条记录，将记录合并在一起
        sz50_stocks.append(rs.get_row_data())
    print(sz50_stocks)
    codelist=[]
    for i in sz50_stocks:
        codelist.append(i[1])
    print(codelist)
    codeStr=getCodeStr(codelist)
    ansdict=parse_stock_data(codeStr)
    print(ansdict)
    # 登出系统
    bs.logout()
    return ansdict

def getCodeStr(codelist):
    # codelist=['sh.600519','sh.601000','sh.601003']
    url = 'http://qt.gtimg.cn/q=';
    num=len(codelist)
    for i in range(num):
        mid='s_'+codelist[i].replace('.','')
        url = url+mid;
        if i!=num-1:
            url=url+','
    response = requests.get(url)
    if response.status_code == 200:
        data = response.text
        return data
    else:
        print(f"Failed to retrieve data. Status code: {response.status_code}")
        return None
    
def parse_stock_data(data_string):
    pattern = r'v_s_sh\d+="(\d)~([^~]+)~(\d+)~([\d.]+)~([-\d.]+)~([-\d.]+)~(\d+)~(\d+)~~([\d.]+)~([^"]+)"'
    matches = re.findall(pattern, data_string)
    result = []
    for match in matches:
        index, name, code, price, change, percent, volume, turnover, _, _, = match
        color=''
        if float(change)>0:
            color='red'
        elif float(change)<0:
            color='green'
        else:
            color='black'
        entry = {
            # "序号": index,
            # "交易所": int(index),
            "名称": str(name),
            "代码": str(code),
            "现价": [str(price),color],
            # "涨跌": float(change),
            "涨跌幅": [str(percent+'%'),color],
            "成交量": str(volume),
            "成交金额": str(turnover),
        }
        result.append(entry)

    return result
def test():
    #### 登陆系统 ####
    lg = bs.login()
    # 显示登陆返回信息
    print('login respond error_code:'+lg.error_code)
    print('login respond  error_msg:'+lg.error_msg)

    #### 获取沪深A股历史K线数据 ####
    # 详细指标参数，参见“历史行情指标参数”章节；“分钟线”参数与“日线”参数不同。“分钟线”不包含指数。
    # 分钟线指标：date,time,code,open,high,low,close,volume,amount,adjustflag
    # 周月线指标：date,code,open,high,low,close,volume,amount,adjustflag,turn,pctChg
    rs = bs.query_history_k_data_plus("sh.600000",
        "date,time,code,open,high,low,close,volume,amount,adjustflag",
        start_date='2017-07-01', end_date='2017-07-31',
        frequency="5", adjustflag="3")
    print('query_history_k_data_plus respond error_code:'+rs.error_code)
    print('query_history_k_data_plus respond  error_msg:'+rs.error_msg)

    #### 打印结果集 ####
    data_list = []
    while (rs.error_code == '0') & rs.next():
        # 获取一条记录，将记录合并在一起
        data_list.append(rs.get_row_data())
    result = pd.DataFrame(data_list, columns=rs.fields)
    print(result)

    #### 登出系统 ####
    bs.logout()
# test()

login success!
[['2023-03-20', 'sh.600010', '包钢股份'], ['2023-03-20', 'sh.600028', '中国石化'], ['2023-03-20', 'sh.600030', '中信证券'], ['2023-03-20', 'sh.600031', '三一重工'], ['2023-03-20', 'sh.600036', '招商银行'], ['2023-03-20', 'sh.600048', '保利发展'], ['2023-03-20', 'sh.600104', '上汽集团'], ['2023-03-20', 'sh.600111', '北方稀土'], ['2023-03-20', 'sh.600196', '复星医药'], ['2023-03-20', 'sh.600276', '恒瑞医药'], ['2023-03-20', 'sh.600309', '万华化学'], ['2023-03-20', 'sh.600346', '恒力石化'], ['2023-03-20', 'sh.600406', '国电南瑞'], ['2023-03-20', 'sh.600436', '片仔癀'], ['2023-03-20', 'sh.600438', '通威股份'], ['2023-03-20', 'sh.600519', '贵州茅台'], ['2023-03-20', 'sh.600585', '海螺水泥'], ['2023-03-20', 'sh.600690', '海尔智家'], ['2023-03-20', 'sh.600745', '闻泰科技'], ['2023-03-20', 'sh.600809', '山西汾酒'], ['2023-03-20', 'sh.600887', '伊利股份'], ['2023-03-20', 'sh.600893', '航发动力'], ['2023-03-20', 'sh.600900', '长江电力'], ['2023-03-20', 'sh.600905', '三峡能源'], ['2023-03-20', 'sh.601012', '隆基绿能'], ['2023-03-20', 'sh.601066', '中信建投'], ['2023-03-20', 'sh.6010

[{'名称': '包钢股份',
  '代码': '600010',
  '现价': ['1.89', 'black'],
  '涨跌幅': ['0.00%', 'black'],
  '成交量': '1710931',
  '成交金额': '32395'},
 {'名称': '中国石化',
  '代码': '600028',
  '现价': ['6.13', 'green'],
  '涨跌幅': ['-0.33%', 'green'],
  '成交量': '788751',
  '成交金额': '48240'},
 {'名称': '中信证券',
  '代码': '600030',
  '现价': ['24.91', 'red'],
  '涨跌幅': ['0.04%', 'red'],
  '成交量': '1639236',
  '成交金额': '407315'},
 {'名称': '三一重工',
  '代码': '600031',
  '现价': ['16.48', 'green'],
  '涨跌幅': ['-3.29%', 'green'],
  '成交量': '1072097',
  '成交金额': '177636'},
 {'名称': '招商银行',
  '代码': '600036',
  '现价': ['34.02', 'green'],
  '涨跌幅': ['-0.38%', 'green'],
  '成交量': '386833',
  '成交金额': '131896'},
 {'名称': '保利发展',
  '代码': '600048',
  '现价': ['14.00', 'red'],
  '涨跌幅': ['1.74%', 'red'],
  '成交量': '589808',
  '成交金额': '82339'},
 {'名称': '上汽集团',
  '代码': '600104',
  '现价': ['15.30', 'green'],
  '涨跌幅': ['-0.20%', 'green'],
  '成交量': '155674',
  '成交金额': '23835'},
 {'名称': '北方稀土',
  '代码': '600111',
  '现价': ['24.92', 'black'],
  '涨跌幅': ['0.00%', 'black'],

In [20]:
import requests


codelist=['sh.600519','sh.601000','sh.601003']
url = 'http://qt.gtimg.cn/q=';
num=len(codelist)
for i in range(num):
    mid='s_'+codelist[i].replace('.','')
    url = url+mid;
    if i!=num-1:
        url=url+','

print(url)
response = requests.get(url)

if response.status_code == 200:
    data = response.text
    print(data)
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")
    # v_sh601003="1~柳钢股份~601003~4.40~4.46~4.49~72591~34605~37986~4.39~272~4.38~1312~4.37~1194~4.36~671~4.35~208~4.40~785~4.41~1853~4.42~374~4.43~131~4.44~615~~20230809155935~-0.06~-1.35~4.50~4.38~4.40/72591/32080131~72591~3208~0.28~-7.22~~4.50~4.38~2.69~112.76~112.76~1.12~4.91~4.01~0.45~-101~4.42~10.49~-4.81~~~1.22~3208.0131~0.0000~0~ ~GP-A~17.65~-6.18~0.00~-15.49~-3.32~4.88~3.29~5.77~8.64~0.69~2562793200~2562793200~-1.36~22.56~2562793200~~~16.71~0.23~~CNY~0~___D__F__N";
    # v_sh601000="1~唐山港~601000~3.50~3.50~3.49~311263~184458~126806~3.49~2043~3.48~10111~3.47~3595~3.46~3009~3.45~6550~3.50~2087~3.51~7055~3.52~8901~3.53~15952~3.54~11986~~20230809155951~0.00~0.00~3.53~3.45~3.50/311263/108503677~311263~10850~0.53~11.45~~3.53~3.45~2.29~207.41~207.41~1.12~3.85~3.15~0.71~-20673~3.49~10.04~12.28~~~0.66~10850.3677~0.0000~0~ ~GP-A~37.80~-4.11~5.71~9.18~8.19~4.29~2.27~-3.05~-2.51~-3.85~5925928614~5925928614~-29.00~45.83~5925928614~~~52.17~0.00~~CNY~0~___D__F__N";

http://qt.gtimg.cn/q=s_sh600519,s_sh601000,s_sh601003
v_s_sh600519="1~贵州茅台~600519~1890.28~0.28~0.01~18523~350631~~23745.66~GP-A";
v_s_sh601000="1~唐山港~601000~3.50~0.00~0.00~311263~10850~~207.41~GP-A";
v_s_sh601003="1~柳钢股份~601003~4.40~-0.06~-1.35~72591~3208~~112.76~GP-A";



In [21]:
import re

def parse_stock_data(data_string):
    pattern = r'v_s_sh\d+="(\d)~([^~]+)~(\d+)~([\d.]+)~([-\d.]+)~([-\d.]+)~(\d+)~(\d+)~~([\d.]+)~([^"]+)"'
    matches = re.findall(pattern, data_string)

    result = []

    for match in matches:
        index, name, code, price, change, percent, volume, turnover, _, _, = match
        entry = {
            "序号": index,
            "交易所": int(index),
            "股票名字": name,
            "股票代码": int(code),
            "当前价格": float(price),
            "涨跌": float(change),
            "涨跌%": float(percent),
            "成交量": int(volume),
            "成交额": int(turnover),
        }
        result.append(entry)

    return result

# Example data string
data_string = """
"v_s_sh600519=\"1~贵州茅台~600519~2075.95~42.95~2.11~38930~794942~~26078.04~GP-A\";"
"v_s_sh601000=\"1~唐山港~601000~3.50~0.00~0.00~311263~10850~~207.41~GP-A\";"
"v_s_sh601003=\"1~柳钢股份~601003~4.40~-0.06~-1.35~72591~3208~~112.76~GP-A\";"
"""

parsed_data = parse_stock_data(data_string)
for entry in parsed_data:
    print(entry)


{'序号': '1', '交易所': 1, '股票名字': '贵州茅台', '股票代码': 600519, '当前价格': 2075.95, '涨跌': 42.95, '涨跌%': 2.11, '成交量': 38930, '成交额': 794942}
{'序号': '1', '交易所': 1, '股票名字': '唐山港', '股票代码': 601000, '当前价格': 3.5, '涨跌': 0.0, '涨跌%': 0.0, '成交量': 311263, '成交额': 10850}
{'序号': '1', '交易所': 1, '股票名字': '柳钢股份', '股票代码': 601003, '当前价格': 4.4, '涨跌': -0.06, '涨跌%': -1.35, '成交量': 72591, '成交额': 3208}


In [14]:
import copy
import baostock as bs
import re
import datetime
import pandas as pd

def convert_to_formatted_datetime(input_string):
    try:
        # 解析输入字符串为datetime对象
        input_datetime = datetime.datetime.strptime(input_string, "%Y%m%d%H%M%S%f")

        # 格式化为目标字符串格式
        output_string = input_datetime.strftime("%H:%M")
        return output_string
    except ValueError:
        return "Invalid input format"

def format_stock_code(stock_code):
    pattern = r'\b\d{6}\b'
    try:
        stock_code= re.search(pattern, stock_code).group()
    except Exception:
        return None
    if re.match(r'^300\d{3}$', stock_code):
        return f"sz.{stock_code}"
    elif re.match(r'^(600|601|603)\d{3}$', stock_code):
        return f"sh.{stock_code}"
    elif re.match(r'^900\d{3}$', stock_code):
        return f"sh.{stock_code}"
    elif re.match(r'^000\d{3}$', stock_code):
        return f"sz.{stock_code}"
    elif re.match(r'^002\d{3}$', stock_code):
        return f"sz.{stock_code}"
    elif re.match(r'^200\d{3}$', stock_code):
        return f"sz.{stock_code}"
    elif re.match(r'^730\d{3}$', stock_code):
        return f"sh.{stock_code}"
    elif re.match(r'^700\d{3}$', stock_code):
        return f"sh.{stock_code}"
    elif re.match(r'^080\d{3}$', stock_code):
        return f"sz.{stock_code}"
    elif re.match(r'^580\d{3}$', stock_code):
        return f"sh.{stock_code}"
    elif re.match(r'^031\d{3}$', stock_code):
        return f"sz.{stock_code}"
    elif re.match(r'^400\d{3}$', stock_code):
        return f"sb.{stock_code}"
    elif re.match(r'^\d{6}$', stock_code):
        return f"sz.{stock_code}"  # 默认将其他六位数字认为是深市股票
    else:
        return None
    
def getStockMinuteInformation(stock,minute):
    # 获取当前时间
    try:
        lg = bs.login()
        stockmid= format_stock_code(stock)
        print(stockmid)
        message={}
        if stockmid==None:
            message["stockname"]=copy.deepcopy(stock)
            rs = bs.query_stock_industry()
            rs = bs.query_stock_basic(code_name=stock)
            stock=rs.get_row_data()[0]
            message["stockid"]=copy.deepcopy(stock)
            print(message)
        else:
            message["stockid"]=copy.deepcopy(stockmid)
            rs = bs.query_stock_industry()
            rs = bs.query_stock_basic(code=stockmid)
            stock=rs.get_row_data()[1]
            message["stockname"]=copy.deepcopy(stock)
            print(message)

        current_date = datetime.date.today()
        # 将时间格式化为所需的字符串格式
        formatted_time = current_date.strftime("%Y-%m-%d")
        # 计算30天前的日期
        days_before = datetime.timedelta(days=1)
        new_date = current_date - days_before
        # 将日期格式化为所需的字符串格式
        formatted_before_date = new_date.strftime("%Y-%m-%d")
        # print(formatted_before_date,formatted_time)
        rs = bs.query_history_k_data_plus(str(message['stockid']),
            "date,time,code,open,high,low,close,volume,amount,adjustflag",
            start_date=formatted_before_date, end_date=formatted_time,
            frequency=str(minute), adjustflag="3")
        # print('query_history_k_data_plus respond error_code:'+rs.error_code)
        # print('query_history_k_data_plus respond  error_msg:'+rs.error_msg)

        #### 打印结果集 ####
        data_list = []
        ans=[]
        while (rs.error_code == '0') & rs.next():
            # 获取一条记录，将记录合并在一起
            data_list.append(rs.get_row_data())
        # result = pd.DataFrame(data_list, columns=rs.fields)
        # print(result)
        for i in range(len(data_list)):
            ans.append([convert_to_formatted_datetime(data_list[i][1]),data_list[i][6],str(float(data_list[i][8])/float(data_list[i][7]))])
        #### 结果集输出到csv文件 ####   
        # result.to_csv("D:\\history_A_stock_k_data.csv", index=False)
        # print(ans)
        #### 登出系统 ####
        bs.logout()
        print(ans)
        return ans,message
    except Exception as e:
        print(e)
        return None,None

getStockMinuteInformation('601003','5')

login success!
sh.601003
{'stockid': 'sh.601003', 'stockname': '柳钢股份'}
logout success!
[['09:35', '4.4700', '4.508475976650202'], ['09:40', '4.4700', '4.468713780918728'], ['09:45', '4.4600', '4.458748723186925'], ['09:50', '4.4700', '4.452498912570683'], ['09:55', '4.4600', '4.4599956812783415'], ['10:00', '4.4800', '4.468725563909774'], ['10:05', '4.4800', '4.462876362792856'], ['10:10', '4.4600', '4.46559670781893'], ['10:15', '4.4600', '4.457825839499146'], ['10:20', '4.4500', '4.453503460207613'], ['10:25', '4.4400', '4.437498272287491'], ['10:30', '4.4300', '4.438321138211382'], ['10:35', '4.4400', '4.4317422920302505'], ['10:40', '4.4400', '4.441875518672199'], ['10:45', '4.4600', '4.462450743173177'], ['10:50', '4.4600', '4.450186859553948'], ['10:55', '4.4600', '4.455905511811023'], ['11:00', '4.4600', '4.460656799259945'], ['11:05', '4.4600', '4.4511111111111115'], ['11:10', '4.4600', '4.455239852398524'], ['11:15', '4.4500', '4.451739130434783'], ['11:20', '4.4500', '4.45349

([['09:35', '4.4700', '4.508475976650202'],
  ['09:40', '4.4700', '4.468713780918728'],
  ['09:45', '4.4600', '4.458748723186925'],
  ['09:50', '4.4700', '4.452498912570683'],
  ['09:55', '4.4600', '4.4599956812783415'],
  ['10:00', '4.4800', '4.468725563909774'],
  ['10:05', '4.4800', '4.462876362792856'],
  ['10:10', '4.4600', '4.46559670781893'],
  ['10:15', '4.4600', '4.457825839499146'],
  ['10:20', '4.4500', '4.453503460207613'],
  ['10:25', '4.4400', '4.437498272287491'],
  ['10:30', '4.4300', '4.438321138211382'],
  ['10:35', '4.4400', '4.4317422920302505'],
  ['10:40', '4.4400', '4.441875518672199'],
  ['10:45', '4.4600', '4.462450743173177'],
  ['10:50', '4.4600', '4.450186859553948'],
  ['10:55', '4.4600', '4.455905511811023'],
  ['11:00', '4.4600', '4.460656799259945'],
  ['11:05', '4.4600', '4.4511111111111115'],
  ['11:10', '4.4600', '4.455239852398524'],
  ['11:15', '4.4500', '4.451739130434783'],
  ['11:20', '4.4500', '4.453493168342356'],
  ['11:25', '4.4500', '4.44963

In [9]:
from datetime import datetime

def convert_to_formatted_datetime(input_string):
    try:
        # 解析输入字符串为datetime对象
        input_datetime = datetime.strptime(input_string, "%Y%m%d%H%M%S%f")

        # 格式化为目标字符串格式
        output_string = input_datetime.strftime("%Y-%m-%d %H:%M")
        return output_string
    except ValueError:
        return "Invalid input format"

# 测试函数
input_string = "20230808094000000"
formatted_datetime = convert_to_formatted_datetime(input_string)
print(formatted_datetime)


2023-08-08 09:40
